In [1]:
#Header files

import cv2
import numpy as np
import os
from PIL import Image
import sqlite3

In [2]:
# FACE DETECTOR


cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceDetect = cv2.CascadeClassifier(cascPathface)
cam = cv2.VideoCapture(0)

while 1  :
    ret,img = cam.read() # generates colour image but we need grayscale image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,1.3,5) #detect and returns coordinates of the faces
    for(x,y,w,h) in faces :
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2) #draws a rectangle on the colour image
    cv2.imshow("face",img) #displays the image on the window 
    if cv2.waitKey(1) == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()






In [7]:
# CREATE DATASET


cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# faceDetect = cv2.CascadeClassifier(cascPathface)
face_detector = cv2.CascadeClassifier(cascPathface)
# For each person, enter one numeric face id
# flag = 0
# d = {}
def insertOrUpdate(face_id,name):
    conn = sqlite3.connect("d:/FaceBase.db")
    cmd = "select * from people where id="+str(face_id)
    cursor = conn.execute(cmd)
    isRecordExist = 0
    for row in cursor : 
        isRecordExist = 1 
    if isRecordExist:
        cmd = "update people set Name = "+str(name)+"where id = " + str(face_id) 
    else:
        cmd = "insert into people(ID,Name) values("+str(face_id)+","+str(name) + ")"
    conn.execute(cmd)
    conn.commit()
    conn.close()
face_id = input('\n enter user id')
name = input("Enter your name")
insertOrUpdate(face_id,name)

#     d[face_id] = face_name
#         flag = 1
#     else:
#         print("Already exists!! pick another one.")
print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0
while(True):
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("D:/dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 20: # Take 30 face sample and stop video
         break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 enter user id1
Enter your name"sid"

 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


In [13]:
## TRAINER



def getImagesWithID(path):
    # from the path, we need all the lists and store it
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faces = []
    IDs = []
    for imagePath in imagePaths:
        faceImg = Image.open(imagePath).convert('L') # to convert colour image to grayscale
        faceNp = np.array(faceImg,'uint8')
        # getting the name of the user and splitting it to get the last element
        ID = int(os.path.split(imagePath)[-1].split('.')[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(faceNp[y:y+h,x:x+w])
            IDs.append(ID)
#         cv2.imshow("training image:",faceNp)
        
    return IDs,faceSamples
#     print(imagePath) 

recognizer =  cv2.face.LBPHFaceRecognizer_create()
path = "D:/dataset/"
cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
detector = cv2.CascadeClassifier(cascPathface)
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
IDs,faceSamples = getImagesWithID(path)
recognizer.train(faceSamples,np.array(IDs))
recognizer.write('d:recognizer/trainingData.yml')


In [8]:
#DETECTOR

cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceDetect = cv2.CascadeClassifier(cascPathface)
cam = cv2.VideoCapture(0)
rec = cv2.face.LBPHFaceRecognizer_create()
rec.read('D:/recognizer/trainingData.yml')
font = cv2.FONT_HERSHEY_SIMPLEX
def getProfile(face_id):
    conn = sqlite3.connect("d:/FaceBase.db")
    cmd = "select * from people where ID ="+str(face_id)
    cursor = conn.execute(cmd)
    profile = None
    for row in cursor :
        profile = row
    conn.close()
    return profile
        
    
while 1  :
    ret,img = cam.read() # generates colour image but we need grayscale image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,1.3,5) #detect and returns coordinates of the faces
    for(x,y,w,h) in faces :
        
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2) #draws a rectangle on the colour image
        
        id_face , conf = rec.predict(gray[y:y+h,x:x+w])
        profile = getProfile(id_face) 
        if profile != None:
            cv2.putText(img,str(profile[1]),(x,y+h+30),font,2,(255,0,0),3)
            cv2.putText(img,str(profile[2]),(x,y+h+60),font,2,(255,0,0),3)
            cv2.putText(img,str(profile[3]),(x,y+h+90),font,2,(255,0,0),3)
            cv2.putText(img,str(profile[4]),(x,y+h+120),font,2,(255,0,0),3)
        else:
            cv2.putText(img,"unrecognised face",(x,y+h),font,2,(255,0,0),5)
    cv2.imshow("face",img) #displays the image on the window 
    if cv2.waitKey(1) == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()




